<a href="https://colab.research.google.com/github/innosoft21/workspace/blob/main/tensorflow_R/%EB%B9%85%EB%B6%84%EA%B8%B0_%EA%B3%B5%EA%B0%9C%EC%98%88%EC%8B%9C_2%EC%9C%A0%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
X_train = read.csv("/content/X_train.csv",header = T, fileEncoding = "UTF-8")
X_test = read.csv("/content/X_test.csv",header = T, fileEncoding = "UTF-8")
y_train = read.csv("/content/y_train.csv",header = T, fileEncoding = "UTF-8")

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“invalid input found on input connection '/content/X_train.csv'”
Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '/content/X_train.csv'”
Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“invalid input found on input connection '/content/X_test.csv'”
Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '/content/X_test.csv'”


In [3]:
# train dataset merge

library(dplyr)

data = X_train %>% left_join(y_train)
head(data)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Joining, by = "cust_id"



cust_id,X,gender
<int>,<lgl>,<int>


In [4]:
# 데이터 분포 확인

summary(data)

# 여자: 0 / 남자: 1
# 남자일 확률 예측

data %>% count(gender)

    cust_id       X               gender   
 Min.   : NA   Mode:logical   Min.   : NA  
 1st Qu.: NA                  1st Qu.: NA  
 Median : NA                  Median : NA  
 Mean   :NaN                  Mean   :NaN  
 3rd Qu.: NA                  3rd Qu.: NA  
 Max.   : NA                  Max.   : NA  

gender,n
<int>,<int>


In [5]:
# 이상치 NA 치환

data$총구매액 = ifelse(data$총구매액 <0, as.numeric(NA), data$총구매액)
data$총구매액 = ifelse(data$총구매액 == 0, 1, data$총구매액)
data$최대구매액 = ifelse(data$최대구매액 <0, as.numeric(NA), data$최대구매액)
data$환불금액 = ifelse(is.na(data$환불금액), 1, data$환불금액)
summary(data)

    cust_id       X               gender    총구매액       최대구매액    
 Min.   : NA   Mode:logical   Min.   : NA   Mode:logical   Mode:logical  
 1st Qu.: NA                  1st Qu.: NA                                
 Median : NA                  Median : NA                                
 Mean   :NaN                  Mean   :NaN                                
 3rd Qu.: NA                  3rd Qu.: NA                                
 Max.   : NA                  Max.   : NA                                
 환불금액      
 Mode:logical  
               
               
               
               
               

In [6]:
# 총구매액/ 최대구매액 / 환불금액 => log 변환

data %>% count(주구매상품, sort=TRUE) #threshold =0.1
data %>% count(주구매지점, sort= TRUE) #threshold = 0.1

ERROR: ignored

In [ ]:
# 데이터 전처리

data$gender = as.factor(data$gender)
data = data %>% select(- cust_id)

In [ ]:
# train-test 분리

library(caret)

idx = createDataPartition(data$gender, p=0.7, list = FALSE)

train_data = data[idx, ]
test_data = data[-idx, ]

In [ ]:
# 데이터 전처리

library(recipes)

recipe = recipe(gender~., data = train_data) %>%
   step_log(contains("액"), base=10) %>%
   step_other(주구매상품, 주구매지점, threshold=0.01) %>%
   step_normalize(내점일수, 내점당구매건수, 구매주기) %>%
   step_impute_knn(all_predictors()) %>%
   prep(training = train_data)

train_preprocessed = bake(recipe, new_data=train_data)
test_preprocessed = bake(recipe, new_data = test_data)

train_preprocessed %>% head
train_preprocessed %>% summary

In [ ]:
# randomForest Model

library(randomForest)

m_rf = randomForest(gender~., data= train_preprocessed)
p_rf = predict(m_rf, test_preprocessed, type="prob")

In [ ]:
# confusionMatrix(test_preprocessed$gender, p_rf)

library(pROC)
p_rf %>% head()

auc(test_preprocessed$gender, p_rf[, 2])

In [ ]:
# randomForest: auc: 0.6589

m_glm = glm(gender~., data= train_preprocessed, family="binomial")
p_glm = predict(m_glm, test_preprocessed)

auc(test_preprocessed$gender, p_glm) # 0.6669